# Mastering TaQL

TaQL is part of [casacore](https://github.com/casacore/casacore), a set of libraries for radio astronomy data processing.

TaQL stands for "Table Query Language", but it can be used also without tables.

## Using this notebook

This notebook is written in Jupyter, but it contains a binding to the TaQL kernel. If you highlight a code cell, you can press **Shift-Enter** to evaluate it in TaQL.

Navigation in Jupyter notebooks can be tricky. If you are in *command mode* (not editing a cell), lots of keyboard shortcuts are active, like `j` and `k` for scrolling, or `a` for inserting a cell. If you want to type in a cell, make sure to be in *edit mode* by checking you see a blinking cursor. Otherwise typing for example "`add`" can have unexpected results.

## TaQL as a calculator

Although it's not the main intended use, you can use TaQL like a regular calculator:

In [57]:
6*7

42


It can do complex numbers as well:

In [58]:
(3+4i)/(8-1i)

(0.307692307692+0.538461538462j)


Also arrays are supported:

In [74]:
[10,34,21,0,-3.14,8]

[ 10.    34.    21.     0.    -3.14   8.  ]


Most functions you expect to work are actually there:

In [76]:
sin(pi()/2)

1.0


In [81]:
stddev([10,34,21,0,-3.14,8])

13.8380369513


## Units

TaQL can handle units:

In [82]:
4km + 3cm

4.00003 km


In [91]:
1km/3h

0.333333333333 km/(h)


In [112]:
sqrt(3km)

Error in TaQL command: 'using style Python SELECT sqrt(3km)'
  Error in select expression: Erronous use of function sqrt - UnitVal::UnitVal Illegal unit dimensions for root

Angles can be given in `h:m:d` or `d:m` format, or in radians or degrees:

In [111]:
4h56m03.5 + 4d12m43.7 + 1 deg - 0.3 rad

1.08276715834 rad


If you want the result in a different unit, append that unit to an expression:

In [118]:
(4h56m03.5 + 4d12m43.7 + 1 deg - 0.3 rad) deg

62.0379883683 deg


Dates can be given in for example ISO 8601 format, even though TaQL predates that standard.
![ISO 8601 was published on 06/05/88 and most recently amended on 12/01/04.](https://imgs.xkcd.com/comics/iso_8601.png "XKCD 1179")

In [115]:
date() - 1981-01-04

12802.0 d


**Exercise**: when was your 10.000th birth-day (in days)?

For high precision dates, use `meas`.

## Measures

The prefix `meas.` is for functions linking to CasaCore's *measures* library. These functions make it possible to convert measures like directions, epochs, and positions from one reference frame to another. And they are very accurate.

In [127]:
meas.epoch("TAI","1-Jul-2015 00:00")-meas.epoch("TAI","30-Jun-2015 23:59")

[ 61.] s


In [24]:
meas.riseset("SUN", datetime(), [6.60417, 52.91692] deg)

[24-Jan-2016/08:16:37, 24-Jan-2016/15:09:02]


In [26]:
meas.riseset("SUN", date(), [6.60417, 52.91692] deg)

[23-Jan-2016/08:18:34, 23-Jan-2016/15:07:53]


In [25]:
meas.riseset("SUN", date()+1, [6.60417, 52.91692] deg)

[24-Jan-2016/08:16:59, 24-Jan-2016/15:09:59]


Or, if you're really interested in where the sun is at every hour today:

In [31]:
meas.azel("SUN", date()+[0:24]h, [6.60417, 52.91692] deg)

[[ 0.1115  -0.99075]
 [ 0.53765 -0.93974]
 [ 0.8934  -0.83539]
 [ 1.17986 -0.6989 ]
 [ 1.41898 -0.54631]
 [ 1.63125 -0.38813]
 [ 1.83175 -0.23181]
 [ 2.03149 -0.08361]
 [ 2.23873  0.0503 ]
 [ 2.45933  0.16324]
 [ 2.69597  0.2481 ]
 [ 2.94684  0.29817]
 [-3.07806  0.30874]
 [-2.8222   0.27873]
 [-2.57758  0.21114]
 [-2.34865  0.11186]
 [-2.1349  -0.01201]
 [-1.93192 -0.15352]
 [-1.73266 -0.30624]
 [-1.52762 -0.46405]
 [-1.30401 -0.62024]
 [-1.04425 -0.76613]
 [-0.72614 -0.88873]
 [-0.33407 -0.96902]] rad



In [16]:
meas.WGSLL("WSRT") deg

[  6.60417  52.91692] deg


In [42]:
meas.epoch("UT1","1-Jul-2015 00:00")-meas.epoch("UT1","30-Jun-2015 23:00")

[ 3600.04164] s


In [36]:
meas.epoch("TAI",1-Jul-2015 00:00, 'UTC')-meas.epoch("TAI",30-Jun-2015 23:00, 'UTC')

[ 3601.] s


In [44]:
meas.epoch("UTC",1-Jul-2015 00:00, 'UTC')-meas.epoch("UTC",30-Jun-2015 23:59:59, 'UTC')

[ 1.] s


The most recent (at time of writing) leap second was in 30 June 2015, and it was [announced](ftp://hpiers.obspm.fr/iers/bul/bulc/bulletinc.49) on 5 January 2015. This is why you should sometimes update your casacore data directory.

In [1]:
meas.epoch("GMT","1-Jul-2015")-meas.epoch("GMT","30-Jun-2015")

Error in TaQL command: 'using style Python SELECT meas.epoch("GMT","1-Jul-2015")-meas.epoch("GMT","30-Jun-2015")'
  Unknown epoch reference type GMT given in a MEAS function

In [2]:
(meas.epoch("UTC",date() - 4-Jan-1981) d

12802.0 d


In [15]:
(1-Jul-1981 - 29-Jun-1981) s

172800.0 s


In [4]:
select DELAY_DIR from ~/projects/tim/tim.MS::FIELD limit 3

Select result of 1 row
DELAY_DIR                     	
[01h37m41.299, +033d09m35.132]


In [5]:
select * from ~/projects/tim/tim.MS::FIELD

Select result of 1 row
DELAY_DIR                     	PHASE_DIR                     	REFERENCE_DIR                 	CODE	FLAG_ROW	NAME  	NUM_POLY	SOURCE_ID	TIME          	LOFAR_TILE_BEAM_DIR    	
[01h37m41.299, +033d09m35.132]	[01h37m41.299, +033d09m35.132]	[01h37m41.299, +033d09m35.132]		False	BEAM_0	0	-1	4871282388.0 s	[ 0.42625  0.57875] rad


In [ ]:
update bla.MS set DATA[!FLAG]=0

In [7]:
select TIME as bla from ~/projects/tim/tim.MS limit 1

Select result of 1 row
bla            	
4871282393.01 s


In [ ]:
select hdms(DELAY_DIR), "bla" as bla from ~/projects/tim/tim.MS::FIELD

In [ ]:
select * from (select DELAY_DIR, "bla" from ~/projects/tim/tim.MS::FIELD)

In [ ]:
select UVW, ANTENNA1 as ant1, ANTENNA2 as ant2, mscal.ant1name(), mscal.ant2name() from (select from ~/projects/tim/tim.MS limit 10)

In [ ]:
6*7

In [ ]:
select meas.azel("SUN", date()+[0:24]h, "WSRT") deg

Notation of angles

Notation of dates

## Tables

In [ ]:
select mean(DATA[FLAG]) as mymean, WEIGHT from ~/projects/tim/tim.MS limit 2

Columns

limit, offset, etc

Storing the output

### Using groupby

## Structure of a Measurement Set

Example with subquery

Example with mscal

## Baseline selection syntax